# Daily update tool
Run once a day after all games are finished

- Scrape data
- Update categories
- Calculate FiFaX
- Scrape video urls of leaders
- Save database
- Tweet top 5s

In [106]:
reset -fs

In [107]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [108]:
from datetime import date, timedelta
from selenium import webdriver
from bs4 import BeautifulSoup
import time, os
import pandas as pd
import numpy as np

Setting date to yesterday (change if target date is not yesterday)

In [109]:
today = date.today()
yesterday = today + timedelta(days = -1)
yesterday_date = '-'.join([str(yesterday.year), str(yesterday.month), str(yesterday.day)])

Setting up the columns of the dataframe

In [110]:
columns = ['date', 't1', 'pitcher', 't2', 'batter', 'inning', 'result', 'pitch_type', 'mph', 
           'rpm', 'vbreak', 'up_down', 'hbreak', 'left_right', 'count']

In [112]:
chromedriver = '/Applications/chromedriver'
os.environ['webdriver.chrome.driver'] = chromedriver
driver = webdriver.Chrome(chromedriver)

/var/folders/4g/z8wcy4nn4vzgq33ts3mh1lm00000gn/T/ipykernel_905/1577013552.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chromedriver)


In [113]:
target_date = yesterday_date
path = 'https://baseballsavant.mlb.com/gamefeed?date=' + target_date + '&chartType=pitch&legendType=pitchName' + \
        '&playerType=pitcher&inning=&count=&pitchHand=&batSide=&descFilter=&ptFilter=&resultFilter=&hf=pitchVelocity#706856'
driver.get(path)
time.sleep(15)
soup = BeautifulSoup(driver.page_source, 'html.parser')

df = pd.DataFrame(columns = columns)
games = soup.find_all('div', {'class': 'game-container step'})
teams_left = soup.find_all('div', {'class':'team-left'})
teams_right = soup.find_all('div', {'class':'team-right'})
# Iterating through each game in the day
for game, team_left, team_right in zip(games, teams_left, teams_right):
    try:
        pitches = game.find('tbody').find_all('tr')
        t1 = team_left.find('div', {'class':'team-name'}).text.replace('\n', '').strip()
        t2 = team_right.find('div', {'class':'team-name'}).text.replace('\n', '').strip()

    except: # Breaks when no more games are found
        print('Break')
        break
    count_b = 0
    count_s = 0
    for pitch in pitches[::-1]: # Iterating through each pitch in the game
        try:
            row = pitch.find_all('span')

            pitcher = row[1].text
            batter = row[3].text
            inning = int(row[7].text)
            result = row[8].text
            pitch_type = row[9].text
            mph = float(row[10].text)
            rpm = int(row[11].text)

            vbreak = int(row[12].text)
            up_down = row[13].text
            hbreak = int(row[14].text)
            l_r = row[15].text
            entry = [target_date, t1, pitcher, t2, batter, inning, result, pitch_type, mph, rpm, vbreak, up_down, hbreak, l_r, (count_b, count_s)]
            df.loc[len(df)] = entry
        except:
            pass
        if ('ball' in result.lower()) or ('pitchout' in result.lower()): # Logic tree to figure out what the ball/strike count is
            count_b += 1
        elif ('strike' in result.lower()) or ('missed bunt' == result.lower()) or ('foul bunt' == result.lower()) or ('tip' in result.lower()):
            count_s += 1
        elif 'foul' in result.lower():
            if count_s < 2:
                count_s += 1
        if (count_b == 4) or (count_s == 3) or ('in play' in result.lower()) or ('hit by pitch' == result.lower()):
            count_b = 0
            count_s = 0

In [114]:
# Distilling categories to the four we care about
result_dict = {'Ball': 'Ball',
               'Foul': 'Foul',
               'Called Strike': 'Strike',
               'Swinging Strike': 'Strike',
               'In play, out(s)': 'Contact',
               'In play, no out': 'Contact',
               'Ball In Dirt': 'Ball',
               'In play, run(s)': 'Contact',
               'Foul Tip': 'Strike',
               'Hit By Pitch': 'Ball',
               'Foul Bunt': 'Strike',
               'Missed Bunt': 'Strike',
               'Pitchout': 'Ball'}

# Distilling pitches into groups we care about
def group_pitches(x):
    if 'Fastball' in x:
        return 'Fastball'
    elif 'Curve' in x:
        return 'Curveball'
    elif ('Knuc' in x) or (x == 'Splitter'):
        return 'Splitter'
    else:
        return x


In [115]:
# Making sure we keep the raw data
df['left_right_raw'] = df['left_right']
df['result_raw'] = df['result']
df['pitch_type_raw'] = df['pitch_type']

# Narrowing down categories
df['left_right'] = df['left_right'].apply(lambda x: x == '←')
df['result'] = df['result'].map(result_dict)
df['pitch_type'] = df['pitch_type'].apply(group_pitches)

In [116]:
pitch_types = ['Fastball', 'Slider', 'Sinker', 'Changeup', 'Curveball', 'Splitter', 'Cutter']

In [117]:
from sklearn.ensemble import RandomForestClassifier
from joblib import load

Loading the pre-trained model

In [118]:
rf_dict = {}
for pitch_type in pitch_types:
    rf_dict[pitch_type] = load(pitch_type + '2.rf')

In [119]:
def calculate_fifax(args):
    p_type, mph, rpm, vbreak, hbreak, is_left = args[0], args[1], args[2], args[3], args[4], args[5]
    X = np.array([mph, rpm, vbreak, hbreak, is_left]).reshape(1,5)
    return rf_dict[p_type].predict_proba(X)[0][0]

In [120]:
# Calculating FiFaX for each pitch thrown
df['fifax'] = df[['pitch_type', 'mph', 'rpm', 'vbreak', 'hbreak', 'left_right']].apply(calculate_fifax, axis = 1)

In [121]:
df.loc[(df.result == 'Strike') & (df.pitch_type == 'Fastball')].sort_values(by = 'fifax', ascending = False).head(5)

,date,t1,pitcher,t2,batter,inning,result,pitch_type,mph,rpm,vbreak,up_down,hbreak,left_right,count,left_right_raw,result_raw,pitch_type_raw,fifax
1646,2022-5-5,Twins,Felix Bautista,Orioles,Gary Sanchez,6,Strike,Fastball,98.0,2329,5,↓,4,True,"(0, 0)",←,Called Strike,4-Seam Fastball,0.811906
224,2022-5-5,Angels,Kutter Crawford,Red Sox,Jared Walsh,8,Strike,Fastball,96.2,2440,7,↓,11,True,"(0, 1)",←,Swinging Strike,4-Seam Fastball,0.782435
333,2022-5-5,Reds,Hunter Greene,Brewers,Omar Narvaez,1,Strike,Fastball,99.8,2420,8,↓,12,True,"(3, 2)",←,Foul Tip,4-Seam Fastball,0.770834
2741,2022-5-5,Cardinals,Jake McGee,Giants,Paul DeJong,7,Strike,Fastball,96.0,2149,11,↓,13,False,"(0, 2)",→,Swinging Strike,4-Seam Fastball,0.751760
1651,2022-5-5,Twins,Felix Bautista,Orioles,Trevor Larnach,6,Strike,Fastball,99.3,2418,8,↓,11,True,"(0, 0)",←,Swinging Strike,4-Seam Fastball,0.751318
370,2022-5-5,Reds,Hunter Greene,Brewers,Rowdy Tellez,2,Strike,Fastball,98.8,2431,8,↓,9,True,"(0, 0)",←,Called Strike,4-Seam Fastball,0.750892
1642,2022-5-5,Twins,Felix Bautista,Orioles,Gio Urshela,6,Strike,Fastball,97.8,2349,7,↓,3,True,"(2, 1)",←,Foul Tip,4-Seam Fastball,0.750415
2226,2022-5-5,Rays,Colin Poche,Mariners,Abraham Toro,7,Strike,Fastball,93.8,2408,8,↓,7,False,"(0, 0)",→,Called Strike,4-Seam Fastball,0.740915
1645,2022-5-5,Twins,Felix Bautista,Orioles,Gio Urshela,6,Strike,Fastball,99.8,2350,4,↓,4,True,"(3, 2)",←,Swinging Strike,4-Seam Fastball,0.731612
306,2022-5-5,Reds,Hunter Greene,Brewers,Luis Urias,1,Strike,Fastball,97.2,2375,8,↓,10,True,"(0, 0)",←,Called Strike,4-Seam Fastball,0.730146


# Scraping baseballsavant for videos

First, need to scrape all player ids

In [122]:
pitcher_dict = {}
batter_dict = {}

Methods to match player names

In [123]:
def comma_name_to_full(comma_name):
    name_list = comma_name.split(', ')
    if len(name_list) == 2:
        return name_list[1] + ' ' + name_list[0]

In [124]:
def delete_suffix(suffix_name):
    suffix_name = suffix_name.replace(' Jr.', '')
    suffix_name = suffix_name.replace(' III', '')
    suffix_name = suffix_name.replace(' II', '')
    return suffix_name

Searches for all players in 2022 season, then grabs their player code

In [125]:
batter_path = f'https://baseballsavant.mlb.com/leaderboard/custom?year={yesterday.year}&type=batter&filter=&sort=1&sortDir=desc&min=1&selections='
driver.get(batter_path)
time.sleep(3)
soup = BeautifulSoup(driver.page_source, 'html.parser')
batters = soup.find('div', {'id':'sortable_stats'}).find('tbody').find_all('tr', {'class':'default-table-row'})

In [126]:
for batter in batters:
    batter_name = batter.find('a').text
    batter_id = batter.find('a')['href'].split('/')[-1]
    batter_dict[delete_suffix(comma_name_to_full(batter_name))] = batter_id

In [127]:
pitcher_path = f'https://baseballsavant.mlb.com/leaderboard/custom?year={yesterday.year}&type=pitcher&filter=&sort=4&sortDir=asc&min=1&selections=p_formatted_ip,exit_velocity_avg,launch_angle_avg,barrel_batted_rate,&chart=false&x=p_formatted_ip&y=p_formatted_ip&r=no&chartType=beeswarm'
driver.get(pitcher_path)
time.sleep(3)
soup = BeautifulSoup(driver.page_source, 'html.parser')
pitchers = soup.find('div', {'id':'sortable_stats'}).find('tbody').find_all('tr', {'class':'default-table-row'})

In [128]:
for pitcher in pitchers:
    pitcher_name = pitcher.find('a').text
    pitcher_id = pitcher.find('a')['href'].split('/')[-1]
    pitcher_dict[delete_suffix(comma_name_to_full(pitcher_name))] = pitcher_id

Searches up the pitch in question, then gets the URL of the raw mp4 video

In [129]:
def get_mp4(leader, driver):
    url = f"https://baseballsavant.mlb.com/statcast_search?hfPT=&hfAB=&hfGT=R%7C&hfPR=&hfZ=&stadium=&hfBBL=&hfNewZones=&hfPull=&hfC={leader['count'][0]}{leader['count'][1]}%7C&hfSea={yesterday.year}%7C&hfSit=&player_type=pitcher&hfOuts=&opponent=&pitcher_throws=&batter_stands=&hfSA=&game_date_gt={yesterday_date}&game_date_lt=&hfInfield=&team=&position=&hfOutfield=&hfRO=&home_road=&batters_lookup%5B%5D={batter_dict[delete_suffix(leader.batter)]}&hfFlag=&hfBBT=&pitchers_lookup%5B%5D={pitcher_dict[delete_suffix(leader.pitcher)]}&metric_1=&hfInn={leader.inning}%7C&min_pitches=0&min_results=0&group_by=name&sort_col=pitches&player_event_sort=api_p_release_speed&sort_order=desc&min_pas=0#results"
    driver.get(url)
    time.sleep(3)
    try:
        driver.find_element_by_class_name('player_name').click()
    except:
        url = f"https://baseballsavant.mlb.com/statcast_search?hfPT=&hfAB=&hfGT=R%7C&hfPR=called%5C.%5C.strike%7Cmissed%5C.%5C.bunt%7Cfoul%5C.%5C.tip%7Cswinging%5C.%5C.pitchout%7Cswinging%5C.%5C.strike%7Cswinging%5C.%5C.strike%5C.%5C.blocked%7C&hfZ=&stadium=&hfBBL=&hfNewZones=&hfPull=&hfC=&hfSea={yesterday.year}%7C&hfSit=&player_type=pitcher&hfOuts=&opponent=&pitcher_throws=&batter_stands=&hfSA=&game_date_gt={yesterday_date}&game_date_lt=&hfInfield=&team=&position=&hfOutfield=&hfRO=&home_road=&batters_lookup%5B%5D={batter_dict[delete_suffix(leader.batter)]}&hfFlag=&hfBBT=&pitchers_lookup%5B%5D={pitcher_dict[delete_suffix(leader.pitcher)]}&metric_1=&hfInn={leader.inning}%7C&min_pitches=0&min_results=0&group_by=name&sort_col=pitches&player_event_sort=api_p_release_speed&sort_order=desc&min_pas=0#results"
        driver.get(url)
        time.sleep(3)
        driver.find_element_by_class_name('player_name').click()
        
    
        
    time.sleep(5)
    try:
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        driver.get('https://baseballsavant.mlb.com' + soup.find('div', {'id':'search-results'}).find('a')['href'])
    except:
        time.sleep(10)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        driver.get('https://baseballsavant.mlb.com' + soup.find('div', {'id':'search-results'}).find('a')['href'])
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    source_url = soup.find('video', {'id':'sporty'}).find('source')['src']
    return source_url

Goes through the FiFaX leaderboard for each pitch type and gets mp4 url of video

In [130]:
df['url'] = 'None'
leaderboard = df.loc[df['result'] == 'Strike'].sort_values(by = 'fifax', ascending = False).head(5)

for i in range(0, len(leaderboard)):
    leader = leaderboard.iloc[i]
    if df.loc[leaderboard.index[i], 'url'] == 'None':
        df.loc[leaderboard.index[i], 'url'] = get_mp4(leader, driver)

/var/folders/4g/z8wcy4nn4vzgq33ts3mh1lm00000gn/T/ipykernel_905/942166042.py:6: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  driver.find_element_by_class_name('player_name').click()


In [133]:
pitch_types = ['Fastball', 'Slider', 'Sinker', 'Changeup', 'Curveball', 'Cutter', 'Splitter']
for pitch_type in pitch_types:
    leaderboard = df.loc[(df['result'] == 'Strike') & (df['pitch_type'] == pitch_type)].sort_values(by = 'fifax', ascending = False).head(5)

    for i in range(0, len(leaderboard)):
        leader = leaderboard.iloc[i]
        if df.loc[leaderboard.index[i], 'url'] == 'None':
            df.loc[leaderboard.index[i], 'url'] = get_mp4(leader, driver)

/var/folders/4g/z8wcy4nn4vzgq33ts3mh1lm00000gn/T/ipykernel_905/942166042.py:6: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  driver.find_element_by_class_name('player_name').click()


For Hitting Samurai

In [135]:
leaderboard = df.loc[(df['result_raw'] == 'In play, no out') | (df['result_raw'] == 'In play, run(s)')].sort_values(by = 'fifax', ascending = False).head(5)

for i in range(0, len(leaderboard)):
    leader = leaderboard.iloc[i]
    df.loc[leaderboard.index[i], 'url'] = get_mp4(leader, driver)

/var/folders/4g/z8wcy4nn4vzgq33ts3mh1lm00000gn/T/ipykernel_905/942166042.py:6: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  driver.find_element_by_class_name('player_name').click()


# Save as CSV, then upload

In [137]:
df.to_csv('daily_push.csv')

In [138]:
pip install google-cloud-storage

Note: you may need to restart the kernel to use updated packages.


In [139]:
from google.cloud import storage

In [140]:
def upload_blob(bucket_name, source_file_name, destination_blob_name):
    
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    
    blob.upload_from_filename(source_file_name)
    
    print('{} with contents {} uploaded to {}'.format(destination_blob_name, source_file_name, bucket_name))
    
    

In [141]:
# Uploading to cloud database
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'google-cloud-auth.json'
upload_blob('the-filthiest', 'daily_push.csv', 'pitch-data/' + target_date + '.csv')

pitch-data/2022-5-5.csv with contents daily_push.csv uploaded to the-filthiest


In [142]:
driver.quit()

# Post to Twitter

In [143]:
!pip install tweepy
import tweepy
import requests
import json


Creating a dict that matches player names to their Twitter handles

In [144]:
class Dict(dict):
        def __missing__(self, key):
            return key
        
def process_twitter_handle(raw_handle):
    output = raw_handle.replace('\n','').split('\t')
    return output        

twitter_handles = open('twitter_handles.txt')
handles = twitter_handles.readlines()


twitter_handle_dict = Dict()
for handle_row in handles:
    name, handle = process_twitter_handle(handle_row)
    twitter_handle_dict[delete_suffix(name)] = handle

In [145]:
js = open('twitter-auth.json')
twitter_auth = json.load(js)


Setting up the Twitter templates for #TheFilthiest and #HittingSamurai

In [156]:
team_handles_hash = {
    'Orioles': ('@Orioles','#Birdland'),
    'Red Sox': ('@RedSox','#DirtyWater'),
    'Rockies': ('@Rockies','#Rockies'),
    'White Sox': ('@WhiteSox','#ChangeTheGame'),
    'Phillies': ('@Phillies','#RingTheBell'),
    'Marlins': ('@Marlins','#MakeItMiami'),
    'Guardians': ('@CleGuardians','#ForTheLand'),
    'Cardinals': ('@Cardinals','#STLCards'),
    'Brewers': ('@Brewers','#ThisIsMyCrew'),
    'Astros': ('@Astros','#LevelUp'),
    'Giants': ('@SFGiants','#SFGameUp'),
    'Mariners': ('@Mariners', '#SeaUsRise'),
    'Blue Jays': ('@BlueJays','#NextLevel'),
    'Cubs': ('@Cubs','#ItsDifferentHere'),
    'Yankees': ('@Yankees','#RepBX'),
    'Rangers': ('@Rangers','#StraightUpTX'),
    'Rays': ('@RaysBaseball','#RaysUp'),
    'Nationals': ('@Nationals','#NATITUDE'),
    'Twins': ('@Twins','#MNTwins'),
    'Angels': ('@Angels','#GoHalos'),
    'Mets': ('@Mets','#LGM'),
    'Pirates': ('@Pirates','#LetsGoBucs'),
    'Padres': ('@Padres','#TimeToShine'),
    'Reds': ('@Reds','#ATOBTTR'),
    'Dbacks': ('@Dbacks',),
    'Tigers': ('@Tigers','#DetroitRoots'),
    'Dodgers': ('@Dodgers','#AlwaysLA'),
    'Royals': ('@Royals','#TogetherRoyal'),
    'Braves': ('@Braves','#ForTheA'),
    "Athletics": ('@Athletics','#DrumTogether')
    
}

number_dict = {1:'1️⃣\U0001F947',
              2:'2️⃣\U0001F948',
              3:'3️⃣\U0001F949',
              4:'4️⃣',
              5:'5️⃣'}

emoji_dict = {1:'\U0001F92E',
              2:'\U0001F631',
              3:'\U0001F974',
              4:'\U0001F62F',
              5:'\U0001F44D'}

def twitter_post(filename, tweet):
    twitter_auth_keys = {
        'bearer_token'        : twitter_auth['twitter_auth']['bearer_token'],
        "consumer_key"        : twitter_auth['twitter_auth']['consumer_key'],
        "consumer_secret"     : twitter_auth['twitter_auth']['consumer_secret'],
        "access_token"        : twitter_auth['twitter_auth']['access_token'],
        "access_token_secret" : twitter_auth['twitter_auth']['access_token_secret']
    }

    client = tweepy.Client(consumer_key=twitter_auth_keys['consumer_key'],
                           consumer_secret=twitter_auth_keys['consumer_secret'],
                           access_token=twitter_auth_keys['access_token'],
                           access_token_secret=twitter_auth_keys['access_token_secret'])
    auth = tweepy.OAuthHandler(
            twitter_auth_keys['consumer_key'],
            twitter_auth_keys['consumer_secret']
            )
    auth.set_access_token(
            twitter_auth_keys['access_token'],
            twitter_auth_keys['access_token_secret']
            )
    api = tweepy.API(auth)
 
    media = api.media_upload(filename, media_category = 'tweet_video')
    #tweet = f"#️⃣{number_dict[rank]} on #TheFilthiest for {yesterday_date}: {pitcher}'s {pitch_type} to {batter} {emoji_dict[rank]}\n\nMPH 🚀: {mph}\nRPM 💫: {rpm}\nFiFaX 🤯: {fifax:.3f}\n\n{team_handles_hash[team1][0]} {team_handles_hash[team1][1]} | {team_handles_hash[team2][0]} {team_handles_hash[team2][1]}"
    status = api.update_status(status=tweet, media_ids = [media.media_id])
 


In [147]:
def pitcher_post(rank, url, pitcher, batter, pitch_type, team1, team2, mph, rpm, fifax, filename):
    tweet = f"#️⃣{number_dict[rank]} on #TheFilthiest for {yesterday_date}: {twitter_handle_dict[pitcher]}'s {pitch_type} to {batter} {emoji_dict[rank]}\n\nMPH 🚀: {mph}\nRPM 💫: {rpm}\nFiFaX 🤯: {fifax:.3f}\n\n{team_handles_hash[team1][0]} {team_handles_hash[team1][1]} | {team_handles_hash[team2][0]} {team_handles_hash[team2][1]}"
    twitter_post(filename, tweet)

def batter_post(rank, url, pitcher, batter, pitch_type, team1, team2, mph, rpm, fifax, filename):
    tweet = f"#️⃣{number_dict[rank]} on #HittingSamurai for {yesterday_date}: {twitter_handle_dict[batter]} prevails against {pitcher}'s {pitch_type} 🗡️\n\nMPH 🚀: {mph}\nRPM 💫: {rpm}\nFiFaX 🤯: {fifax:.3f}\n\n{team_handles_hash[team1][0]} {team_handles_hash[team1][1]} | {team_handles_hash[team2][0]} {team_handles_hash[team2][1]}"
    twitter_post(filename, tweet)

In [148]:
leaderboard = df.loc[df['result'] == 'Strike'].sort_values(by = 'fifax', ascending = False).head(5)
leaderboard

,date,t1,pitcher,t2,batter,inning,result,pitch_type,mph,rpm,vbreak,up_down,hbreak,left_right,count,left_right_raw,result_raw,pitch_type_raw,fifax,url
548,2022-5-5,Reds,Devin Williams,Brewers,Matt Reynolds,8,Strike,Changeup,83.1,2668,41,↓,20,True,"(1, 0)",←,Called Strike,Changeup,0.836440,https://sporty-clips.mlb.com/9fcad6c5-2c0b-4ea...
1646,2022-5-5,Twins,Felix Bautista,Orioles,Gary Sanchez,6,Strike,Fastball,98.0,2329,5,↓,4,True,"(0, 0)",←,Called Strike,4-Seam Fastball,0.811906,https://sporty-clips.mlb.com/f82e4803-e9d6-492...
224,2022-5-5,Angels,Kutter Crawford,Red Sox,Jared Walsh,8,Strike,Fastball,96.2,2440,7,↓,11,True,"(0, 1)",←,Swinging Strike,4-Seam Fastball,0.782435,https://sporty-clips.mlb.com/82797fdd-0bb8-444...
333,2022-5-5,Reds,Hunter Greene,Brewers,Omar Narvaez,1,Strike,Fastball,99.8,2420,8,↓,12,True,"(3, 2)",←,Foul Tip,4-Seam Fastball,0.770834,https://sporty-clips.mlb.com/fbc081a0-a853-443...
120,2022-5-5,Angels,Rich Hill,Red Sox,Brandon Marsh,5,Strike,Slider,68.3,2622,61,↓,22,True,"(3, 2)",←,Swinging Strike,Slider,0.763285,https://sporty-clips.mlb.com/c69c7d96-08f5-4ef...


Given a leaderboard, posting the pitch info and mp4 to Twitter

In [149]:
#countdown = 5
filename = 'upload_vid.mp4'
def post_leaderboard_to_twitter(leaderboard, for_pitcher):
    countdown = len(leaderboard)
    for i in leaderboard.index[::-1]:
        if leaderboard.loc[i,'url'] == 'None':
            pass
        else:
            r = requests.get(leaderboard.loc[i,'url'], stream = True)
            if r.ok:
                print("saving to", filename)
                with open(filename, 'wb') as f:
                    for chunk in r.iter_content(chunk_size=1024 * 8):
                        if chunk:
                            f.write(chunk)
                            f.flush()
                            os.fsync(f.fileno())
            if for_pitcher:
                pitcher_post(countdown, leaderboard.loc[i,'url'], leaderboard.loc[i, 'pitcher'], leaderboard.loc[i,'batter'],
                     leaderboard.loc[i,'pitch_type_raw'], leaderboard.loc[i, 't1'], leaderboard.loc[i, 't2'],
                     leaderboard.loc[i, 'mph'], leaderboard.loc[i, 'rpm'], leaderboard.loc[i, 'fifax'], filename)
            else:
                batter_post(countdown, leaderboard.loc[i,'url'], leaderboard.loc[i, 'pitcher'], leaderboard.loc[i,'batter'],
                     leaderboard.loc[i,'pitch_type_raw'], leaderboard.loc[i, 't1'], leaderboard.loc[i, 't2'],
                     leaderboard.loc[i, 'mph'], leaderboard.loc[i, 'rpm'], leaderboard.loc[i, 'fifax'], filename)
            countdown -= 1
            time.sleep(60)

Post the FiFaX leaderboard that results in a strike, then for reaching safely/scoring runs

In [ ]:
leaderboard = df.loc[df['result'] == 'Strike'].sort_values(by = 'fifax', ascending = False).head(5)
post_leaderboard_to_twitter(leaderboard, True)
leaderboard = df.loc[(df['result_raw'] == 'In play, no out') | (df['result_raw'] == 'In play, run(s)')].sort_values(by = 'fifax', ascending = False).head(5)
post_leaderboard_to_twitter(leaderboard, False)